In [9]:
# Import modules
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
import numpy as np
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet import MobileNet
# Import modules
import tensorflow as tf
import tensorflow.keras as K


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,  Flatten, Dense

import numpy as np
import librosa.display, os

# import keras.utils as image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


In [ ]:
X = np.load('../SavedFeatures/X_mfcc.npy')
y = np.load('../SavedFeatures/y_mfcc.npy')


X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [16]:
def model_mobilenet(input_shape, num_classes):
    base_model = MobileNet(input_shape=input_shape, weights=None, include_top=False)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.5)(x) 
     
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x) 
    x = Dropout(0.4)(x)  
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)  
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)  
    
    x = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs=base_model.input, outputs=x)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
mbnv1 = model_mobilenet(input_shape=(N_MFCC, 109, 1), num_classes=2)

mbnv1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


hist = mbnv1.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.5444 - loss: 0.8139 - val_accuracy: 0.5987 - val_loss: 0.8098
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5977 - loss: 0.7136 - val_accuracy: 0.7008 - val_loss: 0.5901
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.6677 - loss: 0.6309 - val_accuracy: 0.7452 - val_loss: 0.5363
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7446 - loss: 0.5354 - val_accuracy: 0.8033 - val_loss: 0.4477
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7944 - loss: 0.4607 - val_accuracy: 0.8347 - val_loss: 0.3839
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8643 - loss: 0.3465 - val_accuracy: 0.8508 - val_loss: 0.3514
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9024 - loss: 0.2674 - val_accuracy: 0.8614 - val_loss: 0.3439
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9181 - loss: 0.2150 - val_accuracy: 

In [ ]:
test_loss, test_accuracy = mbnv1.evaluate(X_val, y_val, verbose=0)
y_pred = mbnv1.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================MobileNet - MFCC===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")


# Tính EER

eers = []
for i in range(y_pred.shape[1]):  
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
===================MobileNet - MFCC===================
TAccuracy: 0.91048
F1-Score: 0.90818
Class 0: EER = 0.07241 at threshold 0.99610
Class 1: EER = 0.06951 at threshold 0.00393
EER: 0.07096


In [20]:
# Save model as weights in H5 format
mbnv1.save_weights("MFCC_mbnv1.weights.h5")

In [ ]:
del mbnv1, X_train, X_val, y_train, y_val 
gc.collect()


620